# Preprocessing

First we want to load in our function to remove small shapes.

Then we want to run our code which processes the boundaries.

In [3]:
from shapely.geometry import MultiPolygon

def remove_small_shapes(x):
    """
    Remove small multipolygon shapes.

    Parameters
    ---------
    x : polygon
        Feature to simplify.

    Returns
    -------
    MultiPolygon : MultiPolygon
        Shapely MultiPolygon geometry without tiny shapes.

    """
    if x.geometry.type == 'Polygon':
        return x.geometry

    elif x.geometry.type == 'MultiPolygon':

        area1 = 0.003
        area2 = 50

        if x.geometry.area < area1: 
            return x.geometry

        if x['GID_0'] in ['CHL','IDN', 'RUS', 'GRL','CAN','USA']:
            threshold = 0.01
        elif x.geometry.area > area2:
            threshold = 0.1
        else:
            threshold = 0.001

        new_geom = []
        for y in list(x['geometry'].geoms):
            if y.area > threshold:
                new_geom.append(y)

        return MultiPolygon(new_geom)

# Processing national country boundaries

In [4]:
import os
import pandas
import geopandas

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

filename = "gadm36_0.shp"
folder = os.path.join("..", "data", "raw", "gadm36_levels_shp")
path_in = os.path.join(folder, filename)
boundaries = geopandas.read_file(path_in, crs="epsg:4326")

for idx, country_row in countries.iterrows():
    
    if not country_row['iso3'] in ['CAN']:
        continue

    country_boundaries = boundaries[boundaries['GID_0'] == country_row['iso3']]
    
    country_boundaries["geometry"] = country_boundaries.geometry.simplify(
        tolerance=0.01, preserve_topology=True)
        
    country_boundaries['geometry'] = country_boundaries.apply(
        remove_small_shapes, axis=1)
        
    filename = "national_outline.shp"
    folder = os.path.join("..", "data", "processed", country_row['iso3'])
    if not os.path.exists(folder):
        os.makedirs(folder)
    path_out = os.path.join(folder, filename)
    
    country_boundaries.to_file(path_out, crs='epsg:4326')
    
    print("Processed country boundary for {}".format(country_row['iso3']))


D:\Anaconda\envs\ga\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
D:\Anaconda\envs\ga\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
D:\Anaconda\envs\ga\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index

Processed country boundary for CAN


# Processing regional boundaries

In [5]:
import os
import pandas
import geopandas

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

filename = "gadm36_1.shp"
folder = os.path.join("..", "data", "raw", "gadm36_levels_shp")
path_in = os.path.join(folder, filename)
boundaries = geopandas.read_file(path_in, crs="epsg:4326")

for idx, country_row in countries.iterrows():
    
    if not country_row['iso3'] in ['CAN']:
        continue
    
    regions = boundaries[boundaries['GID_0'] == country_row['iso3']]

    regions["geometry"] = regions.geometry.simplify(
        tolerance=0.01, preserve_topology=True)
        
    regions['geometry'] = regions.apply(
        remove_small_shapes, axis=1)

    filename = "regional_shapes_GID_{}.shp".format(country_row['gid_region'])
    folder = os.path.join("..", "data", "processed", country_row['iso3'], "regions")
    if not os.path.exists(folder):
        os.makedirs(folder)
    path_out = os.path.join(folder, filename)
    
    regions.to_file(path_out, crs='epsg:4326')
    
    print("Processed regional boundaries for {}".format(country_row['iso3']))


D:\Anaconda\envs\ga\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
D:\Anaconda\envs\ga\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
D:\Anaconda\envs\ga\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index

Processed regional boundaries for CAN


# Preprocess .tif by cutting to national outline

In [8]:
# Example
import os
import json
import rasterio
from rasterio.mask import mask
import pandas
import geopandas 

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

for idx, country in countries.iterrows():

    if not country['iso3'] == 'CAN': 
        continue
        
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
    path_hazard = os.path.join('..', 'data','raw','flood_hazard', filename)
    hazard = rasterio.open(path_hazard, 'r+')
    hazard.nodata = 255                       #set the no data value
    hazard.crs.from_epsg(4326)                #set the crs
    
    path_country = os.path.join('..', 'data', 'processed', country['iso3'], 'national_outline.shp')
    national_outline = geopandas.read_file(path_country)
    
    geo = geopandas.GeoDataFrame({'geometry': national_outline.geometry})
    coords = [json.loads(geo.to_json())['features'][0]['geometry']]

    out_img, out_transform = mask(hazard, coords, crop=True)
    out_img, out_transform

    out_meta = hazard.meta.copy()
    out_meta.update({"driver": "GTiff",
                "height": out_img.shape[1],
                "width": out_img.shape[2],
                "transform": out_transform,
                "crs": 'epsg:4326'})

    folder = os.path.join('..', 'data', 'processed', country['iso3'], 'hazards', 'inunriver')

    if not os.path.exists(folder):
        os.makedirs(folder)
    path_out = os.path.join(folder, filename)

    with rasterio.open(path_out, "w", **out_meta) as dest:
        dest.write(out_img)

    print('Processing complete.')


Processing complete.


# Convert .tif to shapefile for plotting

In [ ]:
# Example
import os
import json
import rasterio
from rasterio.mask import mask
import pandas
import geopandas 

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

for idx, country in countries.iterrows():
    
    if not country['iso3'] == 'CAN': # if the current country iso3 does not match RWA...
        continue                     # continue in the loop to the next country 
    
    iso3 = country['iso3']
    
    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    if not os.path.exists(folder):
        os.mkdir(folder)
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.shp'
    path_out = os.path.join(folder, filename)

    folder = os.path.join('..', 'data', 'processed', iso3, 'hazards', 'inunriver')
    filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
    path_in = os.path.join(folder, filename)

    with rasterio.open(path_in) as src:

        affine = src.transform
        array = src.read(1)

        output = []

        for vec in rasterio.features.shapes(array):

            if vec[1] > 0 and not vec[1] == 255:

                coordinates = [i for i in vec[0]['coordinates'][0]]

                coords = []

                for i in coordinates:

                    x = i[0]
                    y = i[1]

                    x2, y2 = src.transform * (x, y)

                    coords.append((x2, y2))

                output.append({
                    'type': vec[0]['type'],
                    'geometry': {
                        'type': 'Polygon',
                        'coordinates': [coords],
                    },
                    'properties': {
                        'value': vec[1],
                    }
                })

    output = geopandas.GeoDataFrame.from_features(output, crs='epsg:4326')
    output.to_file(path_out, driver='ESRI Shapefile')


# Plotting

In [51]:
import matplotlib.pyplot as plt
import contextily as cx
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Setup fig and axes, and define plot size
fig, ax = plt.subplots(1, 1, figsize=(8, 6)) 
fig.set_facecolor('gainsboro')

# Make legend accessible and editable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# Import hazard and plot 
folder = os.path.join('..', 'data', 'processed', 'NLD', 'hazards', 'inuncoast')
filename_hazard = 'inuncoast_rcp8p5_wtsub_2080_rp1000_0.shp'
path_in = os.path.join(folder, filename_hazard)
hazard = geopandas.read_file(path_in, crs='epsg:3857') 
hazard = hazard.to_crs(4326)
hazard.plot(column="value", cmap="plasma", legend=True, ax=ax, cax=cax) 

# Import regions and plot 
folder_regions = os.path.join('..', 'data', 'processed', 'NLD')
filename_regions = 'national_outline.shp'
path_in = os.path.join(folder_regions, filename_regions)
regions = geopandas.read_file(path_in, crs='epsg:4326') 
regions = regions.to_crs(4326)
regions.plot(edgecolor='black', facecolor="none", linewidth=.5, legend=True, ax=ax) 

# Add the map baselayer
cx.add_basemap(ax, crs='epsg:4326') 

# Subset scenario strings for title
hazard_type = filename_hazard.split('_')[0]
scenario = filename_hazard.split('_')[1]
model = filename_hazard.split('_')[2]
year = filename_hazard.split('_')[3]
return_period = filename_hazard.split('_')[4]
return_period = return_period.replace('.shp', '')

# Insert scenario strings in title
main_title = 'Projected Coastal Flooding:\n{}, {}, {}, {}, {}, {}'.format(
    'The Netherlands', hazard_type, scenario, model, year, return_period
)
plt.suptitle(main_title, fontsize=16, wrap=True)

# Specify path out
filename_out = 'inuncoast_rcp8p5_wtsub_2080_rp1000_0.png'
path_out = os.path.join('..', 'data', 'processed', 'NLD', filename_out)

# Export image to path
plt.savefig(path_out)  
plt.close()

print('Figure processing complete')

Figure processing complete
